In [1]:
import pandas as pd

In [2]:
data=pd.read_csv("housing.csv")

In [3]:
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [4]:
data=data.drop(columns=['ocean_proximity'])

In [5]:
data.sample(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
9351,-122.55,37.97,52.0,2232.0,291.0,731.0,253.0,7.1155,500001.0
18965,-122.03,38.26,25.0,4617.0,1046.0,2685.0,1011.0,2.9576,108500.0
8384,-118.36,33.95,26.0,3231.0,1089.0,3193.0,1020.0,2.6535,177200.0
11109,-117.90,33.85,32.0,1605.0,314.0,986.0,306.0,3.3375,186200.0
3586,-118.53,34.23,32.0,4039.0,984.0,2675.0,941.0,3.0321,240000.0


In [8]:
data.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
dtype: int64

In [6]:
X=data.drop(columns=['median_house_value'])

In [7]:
Y=data['median_house_value']

In [11]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.ensemble import BaggingRegressor

In [12]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=42)

In [13]:
SI=SimpleImputer(strategy='mean')
X_train_impute=SI.fit_transform(X_train)
X_test_impute=SI.transform(X_test)

In [14]:
X_train_impute = pd.DataFrame(X_train_impute, columns=X_train.columns)
X_test_impute = pd.DataFrame(X_test_impute, columns=X_test.columns)

In [16]:
STD=StandardScaler()
X_train_impute_scaled=STD.fit_transform(X_train_impute)
X_test_impute_scaled=STD.transform(X_test_impute)

## BAGGING REGRESSOR

In [17]:
Bagging=BaggingRegressor(
    estimator=DecisionTreeRegressor(),
    n_estimators=100,
    max_samples=1.0,      # 100% of training data
    max_features=1.0,     # 100% of features
    bootstrap=True,       # WITH replacement (Bagging)
    bootstrap_features=False,
    random_state=42
)

In [20]:
Bagging.fit(X_train_impute_scaled,Y_train)

BaggingRegressor(estimator=DecisionTreeRegressor(), n_estimators=100,
                 random_state=42)

In [21]:
Y_pred=Bagging.predict(X_test_impute_scaled)

In [22]:
r2=r2_score(Y_pred,Y_test)
MSE=mean_squared_error(Y_pred,Y_test)
MAE=mean_absolute_error(Y_pred,Y_test)

In [23]:
print("R2 score is",r2)
print("MSE is",MSE)
print("MAE is",MAE)

R2 score is 0.7578698290523562
MSE is 2487063703.7610993
MAE is 32597.056387273897


## PASTING

In [26]:
Pasting=BaggingRegressor(
    estimator=DecisionTreeRegressor(),
    n_estimators=100,
    max_samples=1.0,
    max_features=1.0,
    bootstrap=False,
    bootstrap_features=False,
    random_state=42
)

In [27]:
Pasting.fit(X_train_impute_scaled,Y_train)

BaggingRegressor(bootstrap=False, estimator=DecisionTreeRegressor(),
                 n_estimators=100, random_state=42)

In [28]:
Y_predp=Pasting.predict(X_test_impute_scaled)

In [29]:
r2p=r2_score(Y_predp,Y_test)
MSEp=mean_squared_error(Y_predp,Y_test)
MAEp=mean_absolute_error(Y_predp,Y_test)

In [30]:
print("R2 score is",r2p)
print("MSE is",MSEp)
print("MAE is",MAEp)

R2 score is 0.6380924745729675
MSE is 4631815183.840033
MAE is 43607.34091731266


## Random Subspaces

In [31]:
Randomsubspaces=BaggingRegressor(
    estimator=DecisionTreeRegressor(),
    n_estimators=100,
    max_samples=1.0,
    max_features=1.0,
    bootstrap=False,
    bootstrap_features=True,
    random_state=42
)

In [32]:
Randomsubspaces.fit(X_train_impute_scaled,Y_train)

BaggingRegressor(bootstrap=False, bootstrap_features=True,
                 estimator=DecisionTreeRegressor(), n_estimators=100,
                 random_state=42)

In [33]:
Y_predR=Randomsubspaces.predict(X_test_impute_scaled)

In [34]:
r2R=r2_score(Y_predR,Y_test)
MSER=mean_squared_error(Y_predR,Y_test)
MAER=mean_absolute_error(Y_predR,Y_test)

In [35]:
print("R2 score is",r2R)
print("MSE is",MSER)
print("MAE is",MAER)

R2 score is 0.7002179627963786
MSE is 2552034735.4266157
MAE is 34846.451682816536


## RandomPatches

In [37]:
Randompatches=BaggingRegressor(
    estimator=DecisionTreeRegressor(),
    n_estimators=100,
    max_samples=1.0,
    max_features=1.0,
    bootstrap=True,
    bootstrap_features=True,
    random_state=42
)

In [38]:
Randompatches.fit(X_train_impute_scaled,Y_train)

BaggingRegressor(bootstrap_features=True, estimator=DecisionTreeRegressor(),
                 n_estimators=100, random_state=42)

In [39]:
Y_predRP=Randompatches.predict(X_test_impute_scaled)

In [40]:
r2RP=r2_score(Y_predRP,Y_test)
MSERP=mean_squared_error(Y_predRP,Y_test)
MAERP=mean_absolute_error(Y_predRP,Y_test)

In [41]:
print("R2 score is",r2RP)
print("MSE is",MSERP)
print("MAE is",MAERP)

R2 score is 0.6813002609846748
MSE is 2559384314.502181
MAE is 35090.546942291134
